<a href="https://colab.research.google.com/github/JINYUHOON/JINYUHOON/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os,sys
from google.colab import drive
drive.mount('/content/drive')
my_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks',my_path)
sys.path.insert(0,my_path)

형태소 : 의미를 가지는 요소로서는 더이상 분석 할 수 없는 가장 작은 말의 단위.
KoNLPy는 시중에 공개된 hannanum, kkma, okt, komoran, mecab 다섯개 형태소 분석기를 한꺼번에 묶어서 편리하게 사용할 수 있도록 한 패키지.

okt
- morphs (phrase, norm=False, stem=False)

Parse phrase to morphemes
- nouns(phrase)
- phrases(phrase)
- pos(phrase, norm=False, stem=False, join=False)
매개변수:
norm == If True, normalize tokens.
stem == If True, stem tokens.
join == If True, returns joined sets of morph and tag

In [ ]:
# 형태소 분석으로 문장을 단어로 분할

from konlpy.tag import Okt

okt = Okt()
print('형태소:',okt.morphs('단독입찰보다 복수입찰의 경우'))
print('명사:',okt.nouns('유일하게 항공기 체계 종합개발 경험을 갖고 있는 KAI는'))
print('구문',okt.phrases('날카로운 분석과 신뢰감 있는 진행으로'))
print('품사',okt.pos('이것도 되나욬ㅋㅋ?'))
print('norm옵션:',okt.pos('이것도 되나욬ㅋㅋ',norm=True))
print('stem옵션:',okt.pos('이것도 되나욬ㅋㅋ',norm=True,stem=True))
print('join옵션:',okt.pos('이것도 되나욬ㅋㅋ',norm=True,stem=True,join=True))

In [ ]:
print('1. 명사만 추출: ',okt.nouns('나는 오늘 방콕에 가고싶다'))
print('2. 원형만 추출: ',okt.pos('나는 오늘 방콕에 갔다',stem=True))
print('3. 형태소 추출: ',okt.morphs('친절한 코치와 재미있는 친구들이 있는 도장에 가고 싶다.'))
print('4. 형태소/태그 추출: ',okt.pos('나는 오늘도 장에 가고 싶다',norm=True,stem=True,join=True))
print('5. 정규화,원형 추출: ',okt.pos('나는 오늘 장에 가고 싶을깤ㅋㅋ?',norm=True,stem=True))

In [ ]:
import re
from konlpy.tag import Okt

okt= Okt()

text = '''김웅 국민의힘 의원이 고위공직자범죄수사처(공수처)의 압수수색 영장을 취소해 달라며 신청한 준항고 사건을 담당할 재판부가 정해졌다.
13일 서울중앙지방법원은 형사31단독(김찬년 판사) 재판부에 김 의원의 준항고 사건을 배당했다고 밝혔다.
준항고는 사법기관의 처분에 불복해 법원에 이의를 제기하는 절차다. 법원이 영장 취소 결정을 내리면 공수처는 영장을 재발부받아야 한다.
김웅 의원 측은 "공수처의 압수수색 영장 집행 과정에서 위법성이 있기 때문에 영장을 취소해달라는 취지의 준항고장을 접수했다"고 밝힌 바 있다.
김 의원 측은 공수처가 김 의원과 변호사의 입회 전에 일부 범죄사실만 언급한 채 영장을 집행하고, 압수물 대상에 적시되지 않은 보좌관과 비서관의 
PC 및 서류를 조사하고 PC 자료 추출 과정에서도 혐의와 관계가 없는 '오수' 등의 검색어로 검색했다는 점을 문제 삼고 있다.
'''

def clean_text(text):
    text = re.sub('[0-9]+','',text)
    text = re.sub('[A-Za-z]+','',text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    return text


text = clean_text(text)
text

In [ ]:
pos_text = okt.pos(text)
noun_list = []
for word, tag in pos_text:
    if tag == 'Noun':
        noun_list.append(word)

print(noun_list)

In [ ]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Okt

fp = codecs.open('/content/drive/MyDrive/cakd3 colab/textmining/dataset/BEXX0003.txt','r',encoding='utf-16')

soup = BeautifulSoup(fp,'html.parser')
body = soup.select_one('body > text')
text = body.getText()

okt = Okt()
word_dic = {}
lines = text.split('\n')
for line in lines:
    malist = okt.pos(line)
    for word in malist:
        if word[1] == 'Noun':
            if not (word[0] in word_dic):
                word_dic[word[0]] = 0
            word_dic[word[0]] += 1 # 카운트하기

keys = sorted(word_dic.items(), key=lambda x:x[1], reverse=True)
for word, count in keys[:50]:
    print('{0}({1})'.format(word,count),end='')

In [ ]:
!pip install --target-$my_path gensim

In [ ]:
import codecs
from bs4 import BeautifulSoup as bs
from konlpy.tag import Okt
from gensim.models import word2vec

okt = Okt()
results =[]
lines= text.split('\n')
for line in lines:
    malist = okt.pos(line, norm=True, stem=True)    # 형태소 분석 # 단어의 기본형 사용
    r=[]
    for word in malist:
        if not word[1] in ['Josa', 'Eomi', 'Punctuation']:  # 어미/조사/구두점 등은 대상에서 제외
            r.append(word[0])
            
    rl= (' '.join(r)).strip()   # " "를 이용해서 r의 리스트 값들을 합치고 좌우의 빈칸 제거
    results.append(rl)
    
# 파일로 출력
wakati_file= '/content/drive/MyDrive/cakd3 colab/textmining/dataset/toji.wakati'
with open(wakati_file,'w', encoding='utf-8') as fp:
    fp.write('\n'.join(results))
    
# W2V 모델 만들기
data = word2vec.LineSentence(wakati_file)
model= word2vec.Word2Vec(data, size=200, window=10,\
                        hs=1, min_count=2, sg=1)
model.save('/content/drive/MyDrive/cakd3 colab/textmining/dataset/toji.model')
print('ok')

In [ ]:
model = word2vec.Word2Vec.load('/content/drive/MyDrive/cakd3 colab/textmining/dataset/toji.model')

In [ ]:
model.wv.most_similar(positive=['집'])